# Protein Complexes in Ovarian Cancer

In [5]:
import pandas as pd
import numpy as np
import re

### Import patient data as well as a list of protein complexes from CORUM (http://mips.helmholtz-muenchen.de/corum/#download 'Complete Complexes')

In [6]:
data = pd.read_csv('proteinGroups_cleaned.txt', sep='\t')
complexData = pd.read_csv('allComplexes.txt', sep='\t')
data

Gene_Name       01OV007   01OV007_NM   01OV008_NM    01OV010_NM  \
0         RBM47  6.632300e+07          0.0          0.0  0.000000e+00   
1          A1CF  6.632300e+07          0.0          0.0  0.000000e+00   
2          UBA6  1.191900e+09          0.0  404400000.0  0.000000e+00   
3         ESYT2  8.069400e+08          0.0          0.0  0.000000e+00   
4         MED19  0.000000e+00          0.0          0.0  0.000000e+00   
5         IGLC7  1.087100e+08          0.0          0.0  0.000000e+00   
6         SHTN1  5.944200e+08          0.0          0.0  0.000000e+00   
7         DAPL1  0.000000e+00          0.0          0.0  0.000000e+00   
8         MEX3A  0.000000e+00          0.0          0.0  0.000000e+00   
9         ILVBL  4.620300e+08  134350000.0          0.0  1.628500e+08   
10        SSC5D  0.000000e+00  122480000.0   90859000.0  0.000000e+00   
11     SH3PXD2B  2.019100e+08          0.0          0.0  0.000000e+00   
12      MACROD2  0.000000e+00          0.0          0.0  0.000000e+00   
13         DYTN  0.000000e+00          0.0          0.0  0.000000e+00   
14         NBAS  3.370800e+07          0.0          0.0  0.000000e+00   
15       TARSL2  0.000000e+00          0.0          0.0  0.000000e+00   
16         VWA8  0.000000e+00          0.0          0.0  0.000000e+00   
17        SBNO1  0.000000e+00          0.0          0.0  0.000000e+00   
18        WDR91  9.334500e+07          0.0          0.0  0.000000e+00   
19        KLRG2  0.000000e+00          0.0          0.0  0.000000e+00   
20        XIRP2  0.000000e+00          0.0          0.0  7.081700e+09   
21       VPS37C  3.224900e+07          0.0          0.0  0.000000e+00   
22      TMEM189  0.000000e+00          0.0          0.0  0.000000e+00   
23        CNOT1  1.321800e+09          0.0          0.0  0.000000e+00   
24     MAP1LC3B  4.291400e+07          0.0          0.0  3.559900e+08   
25    MAP1LC3B2  4.291400e+07          0.0          0.0  3.559900e+08   
26        PALM3  0.000000e+00          0.0          0.0  0.000000e+00   
27          PGP  1.056700e+08          0.0          0.0  6.231300e+08   
28      C5orf51  0.000000e+00          0.0          0.0  0.000000e+00   
29      POM121B  0.000000e+00          0.0          0.0  0.000000e+00   
...         ...           ...          ...          ...           ...   
7004   CHCHD2P9  1.971700e+08          0.0          0.0  0.000000e+00   
7005       EPN1  3.922600e+08          0.0          0.0  0.000000e+00   
7006      PXMP4  0.000000e+00          0.0          0.0  0.000000e+00   
7007     TEX264  0.000000e+00          0.0          0.0  0.000000e+00   
7008      CEPT1  0.000000e+00          0.0          0.0  0.000000e+00   
7009     DNMT3A  0.000000e+00          0.0          0.0  0.000000e+00   
7010       OAS3  9.277000e+08          0.0          0.0  0.000000e+00   
7011      IKBKG  5.996600e+07          0.0          0.0  0.000000e+00   
7012    IGF2BP2  1.701200e+08          0.0          0.0  0.000000e+00   
7013    SLC30A1  0.000000e+00          0.0          0.0  0.000000e+00   
7014     SLC4A7  0.000000e+00          0.0          0.0  0.000000e+00   
7015     NDUFB9  1.583800e+08          0.0          0.0  9.234100e+08   
7016       SQOR  1.156100e+09          0.0          0.0  0.000000e+00   
7017      CAPN6  0.000000e+00          0.0          0.0  0.000000e+00   
7018      AP1M2  0.000000e+00          0.0          0.0  0.000000e+00   
7019      NUMBL  0.000000e+00          0.0          0.0  0.000000e+00   
7020      FCGBP  5.837800e+07          0.0          0.0  0.000000e+00   
7021       SCIN  0.000000e+00          0.0          0.0  0.000000e+00   
7022      DDX49  0.000000e+00          0.0          0.0  0.000000e+00   
7023      CAPN7  0.000000e+00          0.0          0.0  0.000000e+00   
7024      WASF2  2.085700e+08          0.0          0.0  0.000000e+00   
7025       MAU2  0.000000e+00          0.0          0.0  0.000000e+00   
7026    FAM169A  0.000000e+00          0.0          0.0  0.00000

### Create a workable dictionary of protein complex information

In [7]:
# Select for human complexes
complexData = complexData.loc[complexData['Organism'] == 'Human']

# Split the proteins in each complex into invdividual columns of a new df
subunitNames = complexData['subunits(Gene name)'].str.split(';', expand = True)
subunitNames.index = complexData.ComplexName
subunitNames

# Create a dictionary (key = complex name, value = list of proteins in complex) and remove None values
subunitNames = subunitNames.transpose().to_dict('list')
for key, val in subunitNames.items():
    subunitNames[key] = [value for value in val if value != None]
subunitNames

/anaconda3/envs/ptmresearch/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  # Remove the CWD from sys.path while we load stuff.


{'BCL6-HDAC4 complex': ['BCL6', 'HDAC4'],
 'BCL6-HDAC5 complex': ['BCL6', 'HDAC5'],
 'BCL6-HDAC7 complex': ['BCL6', 'HDAC7'],
 'Multisubunit ACTR coactivator complex': ['EP300',
  'CREBBP',
  'KAT2B',
  'NCOA3'],
 'Condensin I complex': ['SMC2', 'NCAPH', 'NCAPD2', 'NCAPG', 'SMC4'],
 'BLOC-3 (biogenesis of lysosome-related organelles complex 3)': ['HPS1',
  'HPS4'],
 'BLOC-2 (biogenesis of lysosome-related organelles complex 2)': ['HPS6',
  'HPS3',
  'HPS5'],
 'MUS81-CDS1 complex': ['CDS1', 'MUS81'],
 'NCOR complex': ['HDAC3', 'TBL1X', 'NCOR1', 'GPS2', 'CORO2A', 'TBL1XR1'],
 'BLOC-1 (biogenesis of lysosome-related organelles complex 1)': ['SNAPIN',
  'BLOC1S1',
  'BLOC1S3',
  'BLOC1S2',
  'BLOC1S5',
  'DTNBP1',
  'BLOC1S4',
  'BLOC1S6'],
 'Arp2/3 protein complex': ['ARPC1B',
  'ARPC2',
  'ARPC3',
  'ARPC5',
  'ARPC4',
  'ACTR3',
  'ACTR2'],
 'PA28gamma complex': ['PSME3'],
 'PA28 complex': ['PSME1', 'PSME2'],
 'PA700 complex': ['PSMD11',
  'PSMD12',
  'PSMD9',
  'PSMD14',
  'PSMD3',
  '

### Define basic analysis functions

In [68]:
def find_ratios(protein_list, patient_id, data):
    patient_normal_id = patient_id + '_NM'
    cancer_total = 0
    normal_total = 0
    cancer_ratios = []
    normal_ratios = []
    for protein in protein_list:
        cancer_total += data.at[protein, patient_id]
        normal_total += data.at[protein, patient_normal_id]
    for protein in protein_list:
        cancer_ratios.append((data.at[protein, patient_id]) / cancer_total)
        normal_ratios.append((data.at[protein, patient_normal_id]) / normal_total)
    return cancer_ratios, normal_ratios